In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/iterative-tensor-decomposition-colab

/content/drive/MyDrive/iterative-tensor-decomposition-colab


In [ ]:
from run.run_tca import decompose
import numpy as np
import scipy.ndimage as spnd
import torch
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = np.load('./data/IBL_neurons_10_warped.npy')/.01

idx  = np.where(np.array([np.mean(d) for d in data]) >.2)[0]
data = data[idx]

data = spnd.gaussian_filter1d(data, sigma=2)
data = np.array([d-np.min(d) for d in data])
data = np.array([d/np.max(d) for d in data])
data = torch.from_numpy(data).float().permute(1, 0, 2).to(device)

In [ ]:
data.shape

torch.Size([831, 221, 350])

In [ ]:
print((1*1*15*860000)/(831*221*350))

0.20069121789232217


In [ ]:
#The hyperparameters
slice_TCA = True
positive = True
orthogonal_constraint  = False
cross_validation = False
orthogonal_penalty = False
metric = False
do_varimax = False
cut_cv_mask = 4

mask_cross_validation = 0.2

learning_rate = 0.02
mask = .8

decay_rate_lr = 1
decay_rate_mask = 0.5
decay_iterations = 5
decay_type_lr='exponential'
decay_type_mask='exponential'

iterations = decay_iterations*99
batch_size = 20
test_freq = -1
initialization = 'uniform-positive'

verbose_train = True
verbose_test = True

In [ ]:
mask_cross_validation = [0]

res = []
for seed in range(10):
    res.append(decompose(seed=seed, data=data, number_components=[2,3,3], iterations=iterations,
                decay_rate_mask=decay_rate_mask, decay_rate_lr=decay_rate_lr, decay_iterations=decay_iterations,
                decay_type_mask=decay_type_mask, decay_type_lr=decay_type_lr, learning_rate=learning_rate, mask=mask,
                mask_cross_validation=mask_cross_validation[0], batch_size=batch_size, sliceTCA=True, positive=positive,
                orthogonal_penalty=orthogonal_penalty, metric=metric, cross_validation=False,
                precision=torch.float32, orthogonal_constraint=orthogonal_constraint, cut_cv_mask=cut_cv_mask,
                initialization=initialization, orthogonal_skip=(), test_freq=-1, verbose_train=verbose_train,
                verbose_test=verbose_test, device=device))

Streaming output truncated to the last 5000 lines.
Iteration: 38 	mse_loss: 0.0580325909
Iteration: 39 	mse_loss: 0.0578261428
Iteration: 40 	mse_loss: 0.0623776466
Test -- Iteration: 40 	mse_loss: 0.0623139478
Iteration: 41 	mse_loss: 0.0622183569
Iteration: 42 	mse_loss: 0.0619361885
Iteration: 43 	mse_loss: 0.0615781173
Iteration: 44 	mse_loss: 0.0611634068
Iteration: 45 	mse_loss: 0.0607211068
Iteration: 46 	mse_loss: 0.0602798946
Iteration: 47 	mse_loss: 0.0598539636
Iteration: 48 	mse_loss: 0.0594529137
Iteration: 49 	mse_loss: 0.0590884052
Iteration: 50 	mse_loss: 0.0587555580
Iteration: 51 	mse_loss: 0.0584507100
Iteration: 52 	mse_loss: 0.0581758209
Iteration: 53 	mse_loss: 0.0579195842
Iteration: 54 	mse_loss: 0.0576813668
Iteration: 55 	mse_loss: 0.0574625246
Iteration: 56 	mse_loss: 0.0572609119
Iteration: 57 	mse_loss: 0.0570730567
Iteration: 58 	mse_loss: 0.0568962432
Iteration: 59 	mse_loss: 0.0567297861
Iteration: 60 	mse_loss: 0.0612825155
Test -- Iteration: 60 	mse_lo

In [ ]:
bestmodel = np.argmin([r[2] for r in res])
components, metric_value, ls, l_cv, convergence, model = res[bestmodel]
pickle.dump(model.cpu(), open('./IBL_sliceTCA-2-3-3.p', 'wb'))

In [ ]:
models = [r[-1].cpu() for r in res]
losses = [r[2] for r in res]

In [ ]:
pickle.dump([models, losses], open('./IBL_2-3-3_all-models.p', 'wb'))